In [1]:
import pyscfad
from pyscfad import gto,dft,scf
import matplotlib.pyplot as plt
import equinox as eqx
import pyscf
# from pyscf import gto,dft,scf
import numpy as np
import jax.numpy as jnp
import scipy
from ase import Atoms
from ase.io import read
import xcquinox as xce
from functools import partial
from ase.units import Bohr
import os, optax, jax
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12030, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
PRETRAIN_LEVEL = 'MGGA'

REFERENCE_XC = 'PBE0'

N_HIDDEN = 16
DEPTH = 3
if PRETRAIN_LEVEL == 'GGA':
    localx = xce.net.eX(n_input=1, n_hidden=N_HIDDEN, use=[1], depth=DEPTH, lob=1.804)
    localc = xce.net.eC(n_input=3, n_hidden=N_HIDDEN, use=[2], depth=DEPTH, ueg_limit=True)
elif PRETRAIN_LEVEL == 'MGGA':
    localx = xce.net.eX(n_input=2, n_hidden=N_HIDDEN, use=[1, 2], depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=4, n_hidden=N_HIDDEN, depth=DEPTH, use=[2,3], ueg_limit=True)
elif PRETRAIN_LEVEL == 'NONLOCAL':
    localx = xce.net.eX(n_input=18, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=16, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True)

xc = xce.xc.eXC(grid_models=[localx, localc], heg_mult=True, level= {'GGA':2, 'MGGA':3, 'NONLOCAL':4}[PRETRAIN_LEVEL],
               verbose=False)

In [3]:
try:
    trainms = read('/home/awills/Documents/Research2/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
except:
    trainms = read('/home/awills/Documents/Research/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []
for idx, at in enumerate(trainms[1:2]):
    name, mol = xce.utils.ase_atoms_to_mol(at, basis='def2tzvpd')
    mol.verbose=9
    mol.build()
    mols.append(mol)
    mf = dft.RKS(mol, xc='SCAN')
    mf.grids.level = 1
    e_tot = mf.kernel()
    mf.conv_tol = 1e-6
    mfs.append(mf)
    dm = mf.make_rdm1()
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=2))
    energies.append(mf.get_veff().exc)
    dms.append(dm)
    ogds.append(dm.shape)
    ao_evals.append(ao_eval)
    gws.append(mf.grids.weights)
    ts.append(mol.intor('int1e_kin'))
    vs.append(mol.intor('int1e_nuc'))
    mo_occs.append(mf.mo_occ)
    hcs.append(mf.get_hcore())
    eris.append(mol.intor('int2e'))
    ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp'))))
    hologaps.append(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1])

System: uname_result(system='Linux', node='raider', release='6.5.0-27-generic', version='#28~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 15 10:51:06 UTC 2', machine='x86_64')  Threads 20
Python 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
numpy 1.26.4  scipy 1.11.4
Date: Mon Apr 22 15:05:25 2024
PySCF version 2.3.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] ARGPARSE = False
[CONFIG] DEBUG = False
[CONFIG] MAX_MEMORY = 4000
[CONFIG] TMPDIR = .
[CONFIG] UNIT = angstrom
[CONFIG] VERBOSE = 3
[CONFIG] conf_file = /home/awills/.pyscf_conf.py
[CONFIG] pyscf_numpy_backend = jax
[CONFIG] pyscf_scipy_backend = jax
[CONFIG] pyscf_scipy_linalg_backend = pyscfad
[CONFIG] pyscfad = True
[CONFIG] pyscfad_ccsd_implicit_diff = True
[CONFIG] pyscfad_scf_implicit_diff = True
[INPUT] verbose = 9
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 14
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry 

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


Padding 0 grids
tot grids = 10360
Drop grids 712
    CPU time for setting up grids      1.53 sec, wall time      0.34 sec
MGGA ni.block_loop; input ao.shape=(10, 9648, 74), weight.shape=(9648,), coords.shape=(9648, 3)
nelec by numeric integration = 13.999768115961864
    CPU time for vxc      2.39 sec, wall time      0.51 sec
E1 = -194.87176405237446  Ecoul = 75.29872351955692  Exc = -13.6939695759968
init E= -109.668655429239
    CPU time for initialize scf      7.89 sec, wall time      1.19 sec
  HOMO = -0.395736495215182  LUMO = -0.051563555594479
  mo_energy =
[-14.23014742 -14.22870336  -1.09512451  -0.52440928  -0.42933735
  -0.42933735  -0.3957365   -0.05156356  -0.05156356   0.08269103
   0.11884868   0.25227581   0.29320761   0.29322936   0.30024136
   0.30024136   0.32834915   0.32834915   0.37032273   0.47119126
   0.47119728   0.48012151   0.48012151   0.4976723    0.53739256
   0.70543023   0.71947461   0.79092803   0.79092803   1.06885566
   1.11414054   1.11428482   1.25

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


MGGA ni.block_loop; input ao.shape=(10, 9648, 74), weight.shape=(9648,), coords.shape=(9648, 3)
nelec by numeric integration = 14.000000096594896
    CPU time for vxc      1.98 sec, wall time      0.14 sec
E1 = -194.418820565452  Ecoul = 74.9345444071956  Exc = -13.634583960640317
cycle= 2 E= -109.520505439321  delta_E= -0.000827  |g|= 0.0191  |ddm|= 0.247
    CPU time for cycle = 2      2.89 sec, wall time      0.23 sec
diis-norm(errvec)=0.0489856
diis-c [-7.96154939e-04  1.61724598e-01  8.38275402e-01]
  HOMO = -0.395320894636477  LUMO = -0.0625049133963386
  mo_energy =
[-14.28545649 -14.28407669  -1.09351971  -0.52562689  -0.44242115
  -0.44242115  -0.39532089  -0.06250491  -0.06250491   0.08815288
   0.14030103   0.2706486    0.28814095   0.28820189   0.30019943
   0.30019943   0.3527347    0.3527347    0.39326077   0.48170868
   0.48185619   0.48851609   0.48851609   0.51423883   0.53900038
   0.71801253   0.72029205   0.80496318   0.80496318   1.0819456
   1.10267995   1.1026896

In [4]:
e_tot, mf.conv_tol

(Array(-109.52058212, dtype=float64), 1e-06)

In [5]:
def generate_network_eval_xc(mf, dm, network):
    def eval_xc(xc_code, rho, ao, gw, coords, spin=0, relativity=0, deriv=1, omega=None, verbose=None):
        # print('custom eval_xc; input rho shape: ', rho.shape)
        if len(rho.shape) == 2:
            rho0 = rho[0] #density
            drho = rho[1:4] #grad_x, grad_y, grad_z
            #laplacian next
            # tau = 0.5*(rho[1] + rho[2] + rho[3])
            tau = rho[-1] # tau
            
            non_loc = jnp.zeros_like(tau)
        print(f'decomposed shapes:\nrho0={rho0.shape}\ndrho={drho.shape}\ntau={tau.shape}\nnon_loc={non_loc.shape}')
        if dm.ndim == 3: # If unrestricted (open-shell) calculation

            # Density
            rho0_a = rho0[0]
            rho0_b = rho0[1]

            # jnp.einsumed density gradient
            gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho[:,0],drho[:,0]), jnp.einsum('ij,ij->j',drho[:,1],drho[:,1])
            gamma_ab = jnp.einsum('ij,ij->j',drho[:,0],drho[:,1])

            # Kinetic energy density
            tau_a, tau_b = tau

            # E.-static
            non_loc_a, non_loc_b = non_loc
        else:
            rho0_a = rho0_b = rho0*0.5
            gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
            tau_a = tau_b = tau*0.5
            non_loc_a=non_loc_b = non_loc*0.5

        # xc-energy per unit particle
        # print(f'EVALUATING GRID MODELS; OPTIONAL PARAMETERS:')
        # try:
        #     print(f'gw.shape={gw.shape}, coor.shape={coor.shape}')
        # except:
        #     print('no externally supplied gw or coor')
        # print('eval_xc eval_grid_models call')
        
        def EXC_exc_vs(x):
            exc = network.eval_grid_models(x, mf=mf, dm=dm, ao=ao, gw=gw, coor=coords)
            Exc = jnp.sum(((rho0_a + rho0_b)*exc[:,0])*gw)
            return Exc, exc

        print(f'eval_xc -> Exc_exc and potentials on grid via autodiff')
        Exc_exc, vs = jax.value_and_grad(EXC_exc_vs, has_aux=True)(jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                                jnp.expand_dims(rho0_b,-1),
                                                jnp.expand_dims(gamma_a,-1),
                                                jnp.expand_dims(gamma_ab,-1),
                                                jnp.expand_dims(gamma_b,-1),
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(tau_a,-1),
                                                jnp.expand_dims(tau_b,-1),
                                                jnp.expand_dims(non_loc_a,-1),
                                                jnp.expand_dims(non_loc_b,-1)],axis=-1))
        Exc, exc = Exc_exc
        
        if jnp.sum(jnp.isnan(exc[:, 0])):
            print('NaNs detected in exc. Number of NaNs: {}'.format(jnp.sum(jnp.isnan(exc[:, 0]))))
            raise Error
        else:
            exc = exc[:, 0]
            
        # print('ao shape: ', ao.shape)
        # print('exc from network evaluation on grid models shape: ', exc.shape)
        # print('vs from network evaluation on grid models shape: ', vs.shape)
        # print('Exc from network evaluation on grid models shape: ', Exc)
        vgf = lambda x: network(x, ao, gw, mf=mf, coor=coords)
        mf.network = xc
        mf.network_eval = vgf

        #vrho; d Exc/d rho, separate spin channels
        vrho = vs[:, 0]+vs[:, 1]
        #vtau; d Exc/d tau, separate spin channels
        vtau = vs[:, 7]+vs[:, 8]
        
        vgamma = jnp.zeros_like(vrho)
        
        vlapl = None
        
        fxc = None #second order functional derivative
        kxc = None #third order functional derivative
        
        print(f'shapes: vrho={vrho.shape}, vgamma={vgamma.shape}')
        return exc, (vrho, vgamma, vlapl, vtau), fxc, kxc
    return eval_xc

In [6]:
mf.mol.spin

0

In [7]:
class E_PySCFAD_loss(eqx.Module):
    def __init__(self):
        '''
        The standard energy loss module, RMSE loss of predicted vs. reference energies.
        '''
        super().__init__()

    def __call__(self, model, mf, inp_dm, ao, gw, ref_en):
        '''
        Computes the energy loss for a given model and associated input density matrix, atomic orbitals on the grid, and grid weights

        Loss is the RMSE energy, so predicted energy can potentially be a jax.Array of SCF guesses.

        :param model: The XC object whose forward pass predicts the XC energy based on the inputs here.
        :type model: xcquinox.xc.eXC
        :param inp_dm: The density matrix to pass into the network for density creation on the grid.
        :type inp_dm: jax.Array
        :param ref_en: The reference energy to take the loss with respect to.
        :type ref_en: jax.Array
        :param ao_eval: Atomic orbitals evaluated on the grid
        :type ao_eval: jax.Array
        :param grid_weights: pyscfad's grid weights for the reference calculation
        :type grid_weights: jax.Array
        :return: The RMSE error.
        :rtype: jax.Array
        '''
        print('generating eval_xc function to overwrite')
        # vgf = lambda x: model(x, ao, gw, mf=mf)
        # mf.network = model
        # mf.network_eval = vgf
        mf.max_memory=16000
        # evxc = generate_network_eval_xc(mf=mf, dm=inp_dm, ao=ao, gw=gw, network=model)
        evxc = generate_network_eval_xc(mf=mf, dm=inp_dm, network=model)
        mf.define_xc_(evxc, xctype='MGGA')
        print('predicting energy...')
        e_pred = mf.kernel()
        print('energy predicted')
        eL = jnp.sqrt( jnp.mean((e_pred-ref_en)**2))
        return eL


In [8]:
cpus = jax.devices(backend='cpu')

In [ ]:
scheduler = optax.exponential_decay(init_value = 1e-2, transition_begin=50, transition_steps=500, decay_rate=0.9)
optimizer = optax.adam(learning_rate = 1e-2)
# optimizer = optax.adam(learning_rate = scheduler)

trainer = xce.train.xcTrainer(model=xc, optim=optimizer, steps=500, loss = E_PySCFAD_loss(), do_jit=False, logfile='log')
# with jax.default_device(cpus[0]):
#     newm = trainer(1, trainer.model, mfs, dms, ao_evals, gws, [-109.52596483])
newm = trainer(1, trainer.model, mfs, dms, ao_evals, gws, [-109.52596483])


Epoch 0
Epoch 0 :: Batch 0/1
generating eval_xc function to overwrite
predicting energy...
Set gradient conv threshold to 0.001
Initial guess from minao.
cond(S) = 198544.46318793358
MGGA ni.block_loop; input ao.shape=(10, 9648, 74), weight.shape=(9648,), coords.shape=(9648, 3)
decomposed shapes:
rho0=(9648,)
drho=(3, 9648)
tau=(9648,)
non_loc=(9648,)
eval_xc -> Exc_exc and potentials on grid via autodiff


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


eX.__call__, rho shape: (2, 9648, 3)
eC.__call__, rho shape: (9648, 4)
shapes: vrho=(9648,), vgamma=(9648,)
nelec by numeric integration = 13.999768115961864
    CPU time for vxc      5.81 sec, wall time      4.11 sec
E1 = -194.87176405237446  Ecoul = 75.29872351955692  Exc = -12.811099413441466
init E= -108.785785266684
    CPU time for initialize scf      8.61 sec, wall time      4.32 sec
  HOMO = 0.12285922093724  LUMO = 0.230632161387276
  mo_energy =
[-11.42105342 -11.41918078  -0.28322057   0.05169132   0.08969063
   0.12285922   0.12285922   0.23063216   0.26460516   0.30309024
   0.30309024   0.3572073    0.39744138   0.39833264   0.43094246
   0.48244096   0.48244096   0.53794762   0.53828572   0.58762582
   0.58762582   0.6536726    0.6536726    0.73482074   0.79789
   0.90437118   1.04050405   1.04050405   1.05903608   1.45322924
   1.4538455    1.51021896   1.62788583   1.62788583   1.80878747
   1.80955893   1.84652972   2.03177813   2.03177813   2.33166753
   2.58128868  

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


MGGA ni.block_loop; input ao.shape=(10, 9648, 74), weight.shape=(9648,), coords.shape=(9648, 3)
decomposed shapes:
rho0=(9648,)
drho=(3, 9648)
tau=(9648,)
non_loc=(9648,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 9648, 3)
eC.__call__, rho shape: (9648, 4)
shapes: vrho=(9648,), vgamma=(9648,)
nelec by numeric integration = 14.000001565018144
    CPU time for vxc      2.46 sec, wall time      0.71 sec
E1 = -199.68790250423186  Ecoul = 81.55918826731494  Exc = -13.403340715690181
cycle= 2 E= -107.933700273032  delta_E= -3.31  |g|= 2.89  |ddm|=   38
    CPU time for cycle = 2      3.79 sec, wall time      0.83 sec
diis-norm(errvec)=6.48498
diis-c [-2.39444889  0.51835612  0.48164388]
  HOMO = -0.244457271687017  LUMO = -0.00729546543685889
  mo_energy =
[-1.23952801e+01 -1.23935422e+01 -8.51263542e-01 -3.43568035e-01
 -3.13409246e-01 -3.13409246e-01 -2.44457272e-01 -7.29546544e-03
  4.04845909e-02  4.04845909e-02  5.01717504e-02  2.18570515e-01
  2.

...

eX.__call__, rho shape: (2, 9648, 3)
eC.__call__, rho shape: (9648, 4)
shapes: vrho=(9648,), vgamma=(9648,)
nelec by numeric integration = 13.999998956534224
    CPU time for vxc      2.54 sec, wall time      0.64 sec
E1 = -188.21279168422504  Ecoul = 68.28296455562798  Exc = -11.830399499851868
cycle= 14 E= -108.161871948874  delta_E= 9.94e-05  |g|= 0.0007  |ddm|= 0.385
    CPU time for cycle = 14      2.87 sec, wall time      0.68 sec
diis-norm(errvec)=0.00082107
diis-c [-1.74022534e-10  8.35878414e-02 -7.33675200e-02  3.31729857e-02
 -7.74952040e-01  1.28919643e+00  4.32022493e-01 -7.94827150e-04
  1.11346392e-02]
  HOMO = -0.10943471727605  LUMO = -0.0924189338614731
  mo_energy =
[-12.28309812 -12.28138433  -0.68290242  -0.20099335  -0.16437421
  -0.16437421  -0.10943472  -0.09241893   0.05744202   0.15814329
   0.15814329   0.23252419   0.32719352   0.32918844   0.33097911
   0.35104992   0.35104992   0.3540912    0.3540912    0.48532409
   0.48622915   0.50827794   0.51015403   

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.sh

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224, 3)
eC.__call__, rho shape: (224, 4)
shapes: vrho=(224,), vgamma=(224,)
MGGA ni.block_loop; input ao.shape=(10, 224, 74), weight.shape=(224,), coords.shape=(224, 3)
decomposed shapes:
rho0=(224,)
drho=(3, 224)
tau=(224,)
non_loc=(224,)
eval_xc -> Exc_exc and potentials on grid via autodiff
eX.__call__, rho shape: (2, 224

Modifications in mldftdat, pyscfad, pyscf, xcquinox. 

xcquinox modifications took place in package repo, so no need to find differences

mldftdat and pyscfad changes occured in package directory, need to find differences for mldftdat to put into xcquinox-cider and a patch for pyscfad

In dft.rks; np.isnan check at line 57, import jax

In dft.numint, anywhere the loop generates subset ao/grids to loop over eval_xc with, tagged with #XCQUINOX MODIFICATION

In [ ]:
jax.clear_backends()

In [ ]:
jax.clear_caches()

In [ ]:
eqx.clear_caches()

In [ ]:
pyscfad.jax.clear_backends()

In [ ]:
pyscfad.jax.clear_caches()

In [ ]:
pyscfad.jax.c